In [ ]:
import os
import statistics
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from dotenv import load_dotenv
from load_dataset import OASISDataset
from vqvae import ResponsiveVQVAE2

load_dotenv()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
DATASET_DATA = {
    'oasis': {
        'channels': 1,
        'dimension': 64
    }
}
dataset = "oasis"
LATENT_DIMENSIONS = (32, 16)

In [ ]:
data_root = os.getenv("SLICES_PATH")

transform = transforms.Compose([
        transforms.Resize(DATASET_DATA[dataset]['dimension']),
        transforms.CenterCrop(DATASET_DATA[dataset]['dimension']),
        transforms.ToTensor(),
        # Don't need to normalize because it is done in __getitem__.
])

data = OASISDataset(data_root, transform=transform)
test_size = len(data) // 20
_, test_data = random_split(data, [len(data) - test_size, test_size], generator=torch.Generator().manual_seed(42))

test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

In [ ]:
########################
# Hyperparameters
########################
epochs = 100

num_hiddens = 128
num_residual_hiddens = 32
num_residual_layers = 2

embedding_dim = 64
num_embeddings = 512
commitment_cost = 0.25
decay = 0.99

learning_rate = 1e-3
########################
model = ResponsiveVQVAE2(DATASET_DATA[dataset]['dimension'], LATENT_DIMENSIONS, DATASET_DATA[dataset]['channels'], num_hiddens,
                         num_residual_layers, num_residual_hiddens, num_embeddings, embedding_dim, decay)
state_dict = torch.load("./results/oasis/vqvae.pth")
model.load_state_dict(state_dict)
model = model.to(device)

In [ ]:
model.eval()

originals = next(iter(test_loader))
originals = originals.to(device)

reconstructions, _ = model(originals)

print(originals.shape)
print(reconstructions.shape)

In [ ]:
# Plot the real images
plt.figure(figsize=(30,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Original Images")
plt.imshow(np.transpose(make_grid(originals, padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Reconstructed Images")
plt.imshow(np.transpose(make_grid(reconstructions, padding=5, normalize=True).cpu(),(1,2,0)))
plt.show()

In [ ]:
from ssim import ssim

batch_ssim = []
for _, batch in enumerate(test_loader):
    originals = next(iter(test_loader))
    originals = originals.to(device)
    reconstructions, _ = model(originals)

    batch_ssim.append(ssim(originals, reconstructions, val_range=1).item())

print(statistics.mean(batch_ssim))

## PixelCNN

In [ ]:
import random
from tqdm import tqdm
from pixel_cnn.prior_models import TopPrior, BottomPrior

BATCH_SIZE = 16

In [ ]:
top_prior = TopPrior()
bottom_prior = BottomPrior()

top_prior.load_state_dict(torch.load("./results/oasis/pixel_cnn_top.pth"))
bottom_prior.load_state_dict(torch.load("./results/oasis/pixel_cnn_bottom.pth"))

top_prior.to(device)
_ = bottom_prior.to(device)

In [ ]:
top_sample = torch.zeros([BATCH_SIZE, 16, 16], dtype=int).to(device)
for row in tqdm(range(top_sample.shape[1])):
    for col in range(top_sample.shape[2]):
        with torch.no_grad():
            logits = top_prior(top_sample)
            probs = torch.softmax(logits[:, :, row, col], -1)
            top_sample.data[:, row, col].copy_(
                probs.multinomial(1).squeeze().data
            )

In [ ]:
bottom_sample = torch.zeros([BATCH_SIZE, 32, 32], dtype=int).to(device)
for row in tqdm(range(bottom_sample.shape[1])):
    for col in range(bottom_sample.shape[2]):
        with torch.no_grad():
            logits = bottom_prior(bottom_sample, top_sample)
            probs = torch.softmax(logits[:, :, row, col], -1)
            bottom_sample.data[:, row, col].copy_(
                probs.multinomial(1).squeeze().data
            )

In [ ]:
images = model.decode_codebook(bottom_sample, top_sample)

# Plot the real images
plt.figure(figsize=(10,7))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(make_grid(images, padding=5, normalize=True).cpu(),(1,2,0)))
plt.show()